## Generating interactive, stand-alone animations
In this notebook, we showcase how to generate interactive, stand-alone and time-dependent animations as html files with PyVista and K3D-jupyter.

In [ ]:
import pyvista as pv
import meshio
import numpy as np
import k3d
import matplotlib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
hexcolor = lambda c: int(matplotlib.colors.to_hex(c)[1:], base=16)


After importing the relevant packages, we read in the first two cardiac cycles of the simulation:

In [ ]:
porous_id = 1
fluid_id = 2
interface_id = 3
d_fac = 10

T = 2
num_steps = int(T*40)
filename = "results_vis.xdmf"
grid = pv.read("mesh/mesh.xdmf")

with meshio.xdmf.TimeSeriesReader(filename) as reader:
    points, cells = reader.read_points_cells()
    for k in range(num_steps):
        t, point_data, cell_data = reader.read_data(k)
        for var, data in point_data.items():
            grid[f"{var}_{k}"] = data
            
time_idx = range(num_steps)
animation_t = np.linspace(0, 10, num_steps)   # slow down the visualization to 10s total

Next, we split the domain in the CSF-filled fluid part and the porous part representing the tissue. Additionally, we clip both to visualize the interior of the domains:

In [ ]:
fluid = grid.extract_cells(grid["subdomains"] == fluid_id)
porous = grid.extract_cells(grid["subdomains"] == porous_id)
outer_surf = grid.extract_surface().clip()

def get_surf(mesh):
    mesh = mesh.extract_geometry()
    mesh.compute_normals(inplace=True, non_manifold_traversal=False, point_normals=False,
                        auto_orient_normals=True)
    return mesh

por_clip = get_surf(porous.clip(crinkle=True))
fluid_clip = fluid.clip()

To visualize the flow field in the CSF-filled domains, we compute arrow glyphs for each time step and scale them with the maximum velocity using PyVista.
Next, we transform the first Pyvista glyph a in K3D object and then replace its *vertices* with a dictionary, mapping the animation time and the new vertex location:

In [ ]:
# generate arrow glyphs for flow
arr_max = max([np.linalg.norm(fluid_clip[f"u_{t}"], axis=1).max() for t in time_idx])
arrows = []
for t in tqdm(time_idx):
    arr = fluid_clip.glyph(orient=f"u_{t}", scale=f"u_{t}",
                         factor=0.1/arr_max, tolerance=.005)
    arr.clear_data()
    arrows.append(arr)
    
k3d_arr = k3d.vtk_poly_data(arrows[10], color=hexcolor("white"), side="double")
k3d_arr.vertices = {animation_t[t]:arrows[t].points for t in time_idx}


Then, we apply a similar procedure to the porous domain, with the addition that we want the colorcoding to represent the total pressure and hence set its time evolution as the *attribute* of the corresponding K3D object and adjust the color range accordingly:

In [ ]:
k3d_por = k3d.vtk_poly_data(por_clip, color_attribute=("phi_0", 0,1))
k3d_por.attribute = {animation_t[t]: por_clip[f"phi_{t}"] for t in time_idx}
k3d_por.vertices = {animation_t[t]: k3d_por.vertices + d_fac*por_clip[f"d_{t}"] for t in time_idx}
k3d_por.color_range = k3d.helpers.minmax(list(k3d_por.attribute.values()))

Finally, we generate a K3D object of the outer surface of the mesh (representing the skull) and add all three component to a K3D plot:

In [ ]:
k3d_surf = k3d.vtk_poly_data(outer_surf, color=hexcolor("deepskyblue"),
                             side="double")
pl = k3d.plot(
    camera_rotate_speed=3,
    camera_zoom_speed=5,
    screenshot_scale=1,
    background_color=hexcolor("white"),
    grid_visible=False,
    camera_auto_fit=True,
    axes_helper=False,
    lighting=2
    )
pl += k3d_por
pl += k3d_arr
pl += k3d_surf
pl.display()
pl.camera = [ 0.39, -0.03, -0.03, -0.05,  0.  ,
             -0.02,  0.09, -0.01,  1.  ]